In [1]:
push!(LOAD_PATH, "..")
using ASE
using MatSciPy
using PyPlot
using PyCall
using TightBinding

### Test toy TB model for Al

In [3]:
# reload("Potentials")
# reload("tbtoymodel")
# reload("TightBinding")
# reload("MatSciPy")
tbm = TightBinding.ToyTB.ToyTBModel(r0=2.5, rcut=6.0)
E = TightBinding.potential_energy(at, tbm)
println("E = ", E)
frc = TightBinding.forces(at, tbm)
println("|f|∞ = ", norm(frc[:], Inf))

E = -15.758671486710963
|f|∞ = 4.40619762898109e-16


In [7]:
set_pbc!(at, [false, false, false])

reload("TightBinding")
tbm = TightBinding.ToyTB.ToyTBModel(r0=2.5, rcut=6.0)
E = TightBinding.potential_energy(at, tbm)
println("E = ", E)
frc = TightBinding.forces(at, tbm)
println("|f|∞ = ", norm(frc[:], Inf))

E = -

2.2402263145143673
|f|∞ = 0.14601367064044007


In [5]:
reload("Potentials")
reload("TightBinding")
reload("tbtoymodel")

set_pbc!(at, [false, false, false])
#set_pbc!(at, [true, true, true])

tbm = TightBinding.ToyTB.ToyTBModel(r0=2.5, rcut=8.0)
X = positions(at)
f = TightBinding.potential_energy(at, tbm)
df = TightBinding.forces(at, tbm)[:]
    println("-----------------------------")
    println("  p | error ")
    println("----|------------------------")
    for p = 2:12
        h = 0.1^p
        dfh = zeros(length(df))
        for n = 1:length(df)
            X[n] += h
            set_positions!(at, X)
            dfh[n] = (TightBinding.potential_energy(at, tbm) - f) / h
            X[n] -= h
        end
    #@printf(" %2d | %1.7e \n", p, norm(df - dfh, Inf))
    @printf(" %2d | %1.7e \n", p, norm(dfh + df, Inf))
    end
println("-----------------------------")


-----------------------------
  p | error 
----|------------------------
  2 | 1.4867346e-03 
  3 | 1.4867270e-04 
  4 | 1.4867263e-05 
  5 | 1.4865886e-06 
  6 | 1.5010215e-07 
  7 | 2.4461734e-08 
  8 | 3.1718760e-07 
  9 | 3.1797765e-06 
 10 | 3.8596093e-05 
 11 | 3.8683979e-04 
 12 | 3.9968029e-03 
-----------------------------


### Test NRL-TB model for Al

In [4]:
at = bulk("Al")#; cubic=true)
at = repeat(at, (1, 2, 1))
X = positions(at)
print(length(at))

2

In [5]:
# TEST NRL-TB for Aluminum FCC
# Test force

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    
BOHR = 0.52917721092 

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)

X = positions(at)
f = TightBinding.potential_energy(at, tbm)
df = TightBinding.forces(at, tbm)[:] # / BOHR
    
println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dfh = zeros(length(df))
    for n = 1:length(df)
        X[n] += h
        set_positions!(at, X)
        dfh[n] = (TightBinding.potential_energy(at, tbm) - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dfh + df, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 3.9513270e-04 
  3 | 3.9401830e-05 
  4 | 3.9391108e-06 
  5 | 3.9409857e-07 
  6 | 4.1020036e-08 
  7 | 2.5032824e-08 
  8 | 1.7763568e-07 
  9 | 2.2220586e-06 
 10 | 2.2646937e-05 
 11 | 2.6157016e-04 
 12 | 2.3931984e-03 
-----------------------------


In [8]:
# TEST NRL-TB for Aluminum FCC
# Test hessian (of eigenvalues)

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)

s=3

tbm.nkpoints = (0,0,0)
K, weight = TightBinding.monkhorstpackgrid(at, tbm)
X = positions(at)
TightBinding.update!(at, tbm)
nlist = TightBinding.NeighbourList(tbm.Rcut, at)
Nneig = 1
for (n, neigs, r, R) in Sites(nlist)
    if length(neigs) > Nneig
        Nneig = length(neigs)
    end
end
eps_n, psi_n = TightBinding.d_eigenstate_k(s, tbm, X, nlist, Nneig, K[:,1])
f = eps_n[:] 

~, Hess_ɛ = TightBinding.hessian_k(X, tbm, nlist, Nneig, K[:,1])
df = reshape(Hess_ɛ[s,:,:,:,:], 3*length(at), 3*length(at))

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dfh = zeros(size(df))
    for n = 1:length(X)
        X[n] += h
        set_positions!(at, X)
        
        TightBinding.update!(at, tbm)
        nlist = TightBinding.NeighbourList(tbm.Rcut, at)
        eps_n, psi_n = TightBinding.d_eigenstate_k(s, tbm, X, nlist, Nneig, K[:,1])
        fh = eps_n[:]
        
        dfh[:,n] = (fh - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dfh - df, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 2.9797294e-03 
  3 | 2.9797806e-04 
  4 | 2.9797810e-05 
  5 | 2.9798046e-06 
  6 | 2.9807753e-07 
  7 | 3.0982161e-08 
  8 | 3.8411311e-08 
  9 | 8.1532003e-07 
 10 | 5.0653925e-06 
 11 | 5.6677931e-05 
 12 | 3.8463572e-04 
-----------------------------


In [17]:
# TEST NRL-TB for Aluminum FCC
# Test hessian (for total energy)

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)

f = TightBinding.forces(at, tbm)[:]
X = positions(at)
Hess = TightBinding.hessian(at, tbm)
df = reshape(Hess, 3*length(at), 3*length(at))

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dfh = zeros(size(df))
    for n = 1:length(X)
        X[n] += h
        set_positions!(at, X)
        fh = TightBinding.forces(at, tbm)[:]        
        dfh[:,n] = (fh - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm( dfh[:] + df[:], Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 9.7554539e-04 
  3 | 9.6720799e-05 
  4 | 9.6638225e-06 
  5 | 9.6630632e-07 
  6 | 9.6901536e-08 
  7 | 1.2941155e-08 
  8 | 4.7184479e-08 
  9 | 2.9143354e-07 
 10 | 1.6653345e-06 
 11 | 2.6829532e-05 
 12 | 3.3306691e-04 
-----------------------------


In [20]:
# TEST NRL-TB for Aluminum FCC
# Test d3E (third order derivative tensor of the total energy)

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)

X = positions(at)
#        X[2,2] += 1.0e-15
#        X[3,2] -= 1.0e-15
#        set_positions!(at, X)
Hess = TightBinding.hessian(at, tbm)
f = reshape(Hess, 3*length(at), 3*length(at))
Tens = TightBinding.d3E(at, tbm)
df = reshape(Tens, 3*length(at), 3*length(at), 3*length(at))

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dfh = zeros(size(df))
    for n = 1:length(X)
        X[n] += h
        set_positions!(at, X)
        Hess = TightBinding.hessian(at, tbm)
        fh = reshape(Hess, 3*length(at), 3*length(at))
        
        dfh[n,:] = (fh - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm( dfh[:] - df[:], Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 9.8529916e-03 
  3 | 8.4250365e-03 
  4 | 8.4250060e-03 
  5 | 8.4250057e-03 
  6 | 8.4250057e-03 
  7 | 8.4250059e-03 
  8 | 8.4250068e-03 
  9 | 8.4250621e-03 
 10 | 8.4253601e-03 
 11 | 8.4351862e-03 
 12 | 8.4546478e-03 
-----------------------------


In [51]:
# test fermi_dirac smearing

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")   

ef = 1.0
beta = 1.0
epsn = 0.3

f = TightBinding.fermi_dirac_d2(ef, beta, epsn)
df = TightBinding.fermi_dirac_d3(ef, beta, epsn)

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:10
    h = 0.1^p 
    dfh = zeros(length(df))
    for n = 1:length(df)
        epsn += h
        fh = TightBinding.fermi_dirac_d2(ef, beta, epsn)
        dfh[n] = (fh - f) / h
        epsn -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dfh - df, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 1.2365594e-03 
  3 | 1.2382382e-04 
  4 | 1.2384032e-05 
  5 | 1.2384210e-06 
  6 | 1.2382920e-07 
  7 | 1.2196280e-08 
  8 | 1.0940495e-09 
  9 | 2.8849625e-08 
 10 | 2.6661526e-08 
-----------------------------


In [10]:
# TEST NRL-TB for Aluminum FCC
# test d_eigenstate : eigenvalues

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    
BOHR = 0.52917721092 

s = 1

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)
K, weight = TightBinding.monkhorstpackgrid(at, tbm)

X = positions(at)
TightBinding.update!(at, tbm)

nlist = TightBinding.NeighbourList(tbm.Rcut, at)
Nneig = 1
for (n, neigs, r, R) in Sites(nlist)
    if length(neigs) > Nneig
        Nneig = length(neigs)
    end
end

eps_n, psi_n = TightBinding.d_eigenstate_k(s, tbm, X, nlist, Nneig, K[:,1])

f = TightBinding.get_k_array(tbm, :epsn, K[:,1])[s]
df = eps_n[:]

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dfh = zeros(length(df))
    for n = 1:length(df)
        X[n] += h
        set_positions!(at, X)
        TightBinding.update!(at, tbm)
        fh = TightBinding.get_k_array(tbm, :epsn, K[:,1])[s]
        dfh[n] = (fh - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dfh - df, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 1.5930766e-03 
  3 | 1.5576992e-04 
  4 | 1.5541830e-05 
  5 | 1.5538196e-06 
  6 | 1.5564084e-07 
  7 | 1.6554841e-08 
  8 | 1.6653345e-08 
  9 | 2.5295189e-07 
 10 | 1.6344273e-06 
 11 | 3.1332893e-05 
 12 | 3.3386867e-04 
-----------------------------


In [9]:
# TEST NRL-TB for Aluminum FCC
# test d_eigenstate : eigenfunctions

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    
BOHR = 0.52917721092 

s = 1

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)
K, weight = TightBinding.monkhorstpackgrid(at, tbm)

X = positions(at)
TightBinding.update!(at, tbm)
eps = TightBinding.get_k_array(tbm, :epsn, K[:,1])

nlist = TightBinding.NeighbourList(tbm.Rcut, at)
Nneig = 1
for (n, neigs, r, R) in Sites(nlist)
    if length(neigs) > Nneig
        Nneig = length(neigs)
    end
end

eps_n, psi_n = TightBinding.d_eigenstate_k(s, tbm, X, nlist, Nneig, K[:,1])

f = TightBinding.get_k_array(tbm, :C, K[:,1])[:,s]
df = reshape(psi_n, 3*length(at), length(eps))'

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dfh = zeros(size(df))
    for n = 1:size(df)[2]
        X[n] += h
        set_positions!(at, X)
        TightBinding.update!(at, tbm)
        fh = TightBinding.get_k_array(tbm, :C, K[:,1])[:,s]
        dfh[:,n] = (fh - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(( dfh - df )[:], Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 1.3503005e+02 
  3 | 1.7916602e-04 
  4 | 1.7902184e-05 
  5 | 1.7900218e-06 
  6 | 1.7952114e-07 
  7 | 1.5319154e-08 
  8 | 7.3291674e-08 
  9 | 6.1528752e-07 
 10 | 1.3937964e-05 
 11 | 6.9539720e-05 
 12 | 8.7991192e-04 
-----------------------------


In [9]:
# TEST NRL-TB for Aluminum FCC
# WITH PERIODIC BOUNDARY CONDITION ON THIRD DIMENSION
# TAKE nkpoints = (0,0,4)

reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")    
BOHR = 0.52917721092 

set_pbc!(at, [true, true, true])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (2,8,2)


X = positions(at)
f = TightBinding.potential_energy(at, tbm)
df = TightBinding.forces(at, tbm)[:] # / BOHR
    println("-----------------------------")
    println("  p | error ")
    println("----|------------------------")
for p = 2:10
    h = 0.1^p 
    dfh = zeros(length(df))
    for n = 1:length(df)
        X[n] += h
        set_positions!(at, X)
        dfh[n] = (TightBinding.potential_energy(at, tbm) - f) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dfh + df, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 8.0597075e-04 
  3 | 8.0563563e-05 
  4 | 8.0563202e-06 
  5 | 8.0566677e-07 
  6 | 8.0383845e-08 
  7 | 8.8819055e-09 
  8 | 2.2204055e-07 
  9 | 2.2204443e-06 
 10 | 1.3322688e-05 
-----------------------------


## TEST site energy and site force

In [63]:
at = bulk("Al")#; cubic=true)
at = repeat(at, (1, 2, 2))
X = positions(at)
# set_pbc!(at, [false, false, false])
# plot3D(X[1,:][:], X[2,:][:], X[3,:][:], "b.")
print(length(at))

4

In [64]:
# TEST NRL-TB site energy for Aluminum FCC
# WITH open BOUNDARY CONDITION ON THIRD DIMENSION

using AtomsInterface
reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")  

set_pbc!(at, [false, false, false])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)

X = positions(at)

Es_all = TightBinding.site_energy([1:length(at);], at, tbm)
Etot = TightBinding.potential_energy(at, tbm)
println("E - ∑ E_i = ", Etot - r_sum(Es_all))
println("------------------------------")

println("Finite-difference test")
Es = TightBinding.site_energy(1, at, tbm)
Fs = TightBinding.site_forces(1, at, tbm)[:]

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dEsh = zeros(length(Fs))
    for n = 1:length(Fs)
        X[n] += h
        set_positions!(at, X)
        dEsh[n] = (TightBinding.site_energy(1, at, tbm) - Es) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dEsh + Fs, Inf))
end
println("-----------------------------")

E - ∑ E_i = 5.329070518200751e-15
------------------------------
Finite-difference test
-----------------------------
  p | error 
----|------------------------
  2 | 1.8566967e-04 
  3 | 1.8596655e-05 
  4 | 1.8600000e-06 
  5 | 1.8629885e-07 
  6 | 2.0875614e-08 
  7 | 3.4353978e-08 
  8 | 2.5728062e-07 
  9 | 3.7453059e-06 
 10 | 2.9216484e-05 
 11 | 2.1344333e-04 
 12 | 3.4997035e-03 
-----------------------------


In [7]:
# TEST NRL-TB site energy for Aluminum FCC
# WITH periodic BOUNDARY CONDITION ON THIRD DIMENSION

using AtomsInterface
reload("SparseTools")
reload("Potentials")
reload("TightBinding")
reload("NRLTB")  

set_pbc!(at, [false, true, true])
tbm = NRLTB.NRLTBModel(elem = NRLTB.Al_spd)
tbm.nkpoints = (0,0,0)

X = positions(at)

Es_all = TightBinding.site_energy([1:length(at);], at, tbm)
Etot = TightBinding.potential_energy(at, tbm)
println("E - ∑ E_i = ", Etot - r_sum(Es_all))
println("------------------------------")

println("Finite-difference test")
Es = TightBinding.site_energy(1, at, tbm)
Fs = TightBinding.site_forces(1, at, tbm)[:]

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    h = 0.1^p 
    dEsh = zeros(length(Fs))
    for n = 1:length(Fs)
        X[n] += h
        set_positions!(at, X)
        dEsh[n] = (TightBinding.site_energy(1, at, tbm) - Es) / h
        X[n] -= h
    end
    @printf(" %2d | %1.7e \n", p, norm(dEsh + Fs, Inf))
end
println("-----------------------------")

E - ∑ E_i = 0.0
------------------------------
Finite-difference test
-----------------------------
  p | error 
----|------------------------
  2 | 1.0063240e-03 
  3 | 1.0064698e-04 
  4 | 1.0064709e-05 
  5 | 1.0063949e-06 
  6 | 9.9476247e-08 
  7 | 2.2204517e-08 
  8 | 2.6645355e-07 
  9 | 1.7763569e-06 
 10 | 1.3322676e-05 
 11 | 3.1086245e-04 
 12 | 2.2204460e-03 
-----------------------------


### Test ForwardDiff for dH, d2H, and d3H

In [3]:
a = rand(3)*2

3-element Array{Float64,1}:
 1.27341
 1.769  
 0.33543

In [41]:
# TEST the ForwardDiff for NRL-TB hamiltonian derivatives
# C

import NRLTB
reload("NRLTB")
BOHR = 0.52917721092 

h = zeros(4,4)
f = zeros(4,4)
dh = zeros(3,4,4)
# f = NRLTB.mat_local_h!(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, h, zeros(4))
NRLTB.mat_local_h!(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, h, zeros(4))
f[:] = h[:]
df = NRLTB.hop_local_fd!(a/BOHR, NRLTB.C_sp, dh) / BOHR

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:3
        a[n] += δ
        fh = NRLTB.mat_local_h!(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, h, zeros(4))
        # print(fh-f) println("\n")
        dfh[n,:] = ( (fh - f) / δ ) 
        a[n] -= δ
    end
    err = reshape( dfh - df, 3*4*4)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 1.8473380e-03 
  3 | 1.8703860e-04 
  4 | 1.8727011e-05 
  5 | 1.8729496e-06 
  6 | 1.8729522e-07 
  7 | 1.9679301e-08 
  8 | 2.3439385e-08 
  9 | 2.8418994e-07 
 10 | 9.7608153e-07 
 11 | 1.8793833e-05 
 12 | 2.1548997e-04 
-----------------------------


In [45]:
@time NRLTB.d_mat_local(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, :dH);
@time NRLTB.hop_local_fd!(a/BOHR, NRLTB.C_sp, dh);
@time NRLTB.d_mat_local(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, :dH);
@time NRLTB.hop_local_fd!(a/BOHR, NRLTB.C_sp, dh);
@time NRLTB.d_mat_local(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, :dH);
@time NRLTB.hop_local_fd!(a/BOHR, NRLTB.C_sp, dh);

  0.000011 seconds (28 allocations: 1.656 KB)
  0.000614 seconds (3.25 k allocations: 147.781 KB)
  0.000007 seconds (28 allocations: 1.656 KB)
  0.000547 seconds (3.25 k allocations: 147.781 KB)
  0.000005 seconds (28 allocations: 1.656 KB)
  0.000561 seconds (3.25 k allocations: 147.781 KB)


In [4]:
# TEST the ForwardDiff for NRL-TB hamiltonian derivatives
# Al

import NRLTB
reload("NRLTB")
BOHR = 0.52917721092 

h = zeros(9,9)
f = zeros(9,9)
dh = zeros(3,9,9)
# f = NRLTB.mat_local_h!(norm(a)/BOHR, a/BOHR, NRLTB.C_sp, h, zeros(10))
NRLTB.mat_local_h!(norm(a)/BOHR, a/BOHR, NRLTB.Al_spd, h, zeros(10))
f[:] = h[:]
df = NRLTB.hop_local_fd!(a/BOHR, NRLTB.Al_spd, dh)
df[:] = dh[:]/BOHR

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:3
        a[n] += δ
        fh = NRLTB.mat_local_h!(norm(a)/BOHR, a/BOHR, NRLTB.Al_spd, h, zeros(10))
        # print(fh-f) println("\n")
        dfh[n,:] = ( (fh - f) / δ )
        a[n] -= δ
    end
    err = reshape( dfh - df, 3*9*9)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 4.9542611e-02 
  3 | 5.0066614e-03 
  4 | 5.0119324e-04 
  5 | 5.0124593e-05 
  6 | 5.0125230e-06 
  7 | 4.9885576e-07 
  8 | 1.7522575e-07 
  9 | 1.9910464e-06 
 10 | 1.1976896e-05 
 11 | 8.6725733e-05 
 12 | 1.0261656e-03 
-----------------------------


In [8]:
# test ForwardDiff for hessian
import NRLTB
reload("NRLTB")
BOHR = 0.52917721092 

norb = 9
# b = a/BOHR
dh = zeros(3,3,norb,norb)

h = zeros(3,norb,norb)
f = zeros(3,norb,norb)
fh = zeros(3,norb,norb)
df = zeros(3,3,norb,norb)

#h = NRLTB.hop_local_fd!(a/BOHR, NRLTB.Al_spd, h)
#f[:] = h[:]/BOHR
NRLTB.d_mat_local!(norm(a)/BOHR, a/BOHR, NRLTB.Al_spd, :dH, h)
f[:] = h[:]/BOHR
#NRLTB.evaluate_fd!(tbm.hop, a, f)
# NRLTB.hop_local_fd2!(a/BOHR, NRLTB.Al_spd, df)
# df[:] /= BOHR^2
NRLTB.evaluate_fd2!(tbm.hop, a, df)

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:3
        a[n] += δ
        # fh = NRLTB.hop_local_fd!(a/BOHR, NRLTB.Al_spd, h)
        # fh[:] = h[:]/BOHR
        NRLTB.evaluate_fd!(tbm.hop, a, fh)
        dfh[n,:] = ( (fh - f) / δ ) 
        a[n] -= δ
    end
    err = reshape( dfh - df, 3*3*norb*norb)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 1.7442464e-01 
  3 | 1.7567390e-02 
  4 | 1.7579878e-03 
  5 | 1.7581102e-04 
  6 | 1.7585501e-05 
  7 | 1.7976852e-06 
  8 | 7.1410753e-07 
  9 | 2.3495870e-06 
 10 | 3.8106419e-05 
 11 | 3.0351238e-04 
 12 | 5.6508570e-03 
-----------------------------


In [12]:
# a = [ -2.025, -2.0250000000170014, 0.0 ] 

# note that when x==y or z==0,
# Forwardiff gives the wrong results for 3rd order derivatives for Aluminimu (norb=9)
a = [ -2.025, -2.025+1.0e-10, 1.0e-10 ] 
# a = [ -2.025, -2.025, 0.0 ] 
a = [3.8266954022442503,1.0e-15,3.82669540224614]

3-element Array{Float64,1}:
 3.8267 
 1.0e-15
 3.8267 

In [15]:
# test 3rd order derivatives
import NRLTB
reload("NRLTB")
BOHR = 0.52917721092 

norb = 9
b = a/BOHR

h = zeros(3,3,norb,norb)
f = zeros(3,3,norb,norb)
dh = zeros(3,3,3,norb,norb)

df = NRLTB.hop_local_fd3!(a/BOHR, NRLTB.Al_spd, dh)
df[:] = df[:]/BOHR^3

NRLTB.hop_local_fd2!(a/BOHR, NRLTB.Al_spd, h)
f[:] = h[:]/BOHR^2

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:3
        a[n] += δ
        fh = NRLTB.hop_local_fd2!(a/BOHR, NRLTB.Al_spd, h)
        fh[:] = fh[:]/BOHR^2
        dfh[n,:] = ( (fh - f) / δ )
        a[n] -= δ
    end
    err = reshape( dfh - df, 3*3*3*norb*norb)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 5.8843486e-05 
  3 | 5.8654554e-06 
  4 | 5.8635744e-07 
  5 | 5.8633407e-08 
  6 | 5.8702741e-09 
  7 | 6.0625572e-10 
  8 | 1.2524250e-09 
  9 | 1.3752918e-08 
 10 | 1.6901067e-07 
 11 | 1.2771942e-06 
 12 | 1.5078959e-05 
-----------------------------


In [75]:
@time  NRLTB.hop_local_fd3!(a/BOHR, NRLTB.Al_spd, dh);
@time  NRLTB.hop_local_fd3!(a/BOHR, NRLTB.Al_spd, dh);

  0.017657 seconds (58.87 k allocations: 3.568 MB)
  0.013645 seconds (58.87 k allocations: 3.568 MB)


### ForwardDiff  for onsite  terms

In [44]:
r = rand(3,5)*4

3x5 Array{Float64,2}:
 1.69863   3.57175   3.99857    1.30786   3.7234 
 3.7917    0.972753  0.0575383  3.80522   3.7646 
 0.372574  1.86909   0.667562   0.810392  2.52693

In [16]:
R = Float64[ norm(r[:,k]) for k = 1:size(r)[2]]

5-element Array{Float64,1}:
 4.18777
 5.05741
 3.76347
 4.37487
 3.65041

In [17]:
import NRLTB
reload("NRLTB")

h = NRLTB.get_os(R, NRLTB.C_sp)
d3h = zeros(15, 15, 15, 4)
df = NRLTB.get_d3os_fd!(r, NRLTB.C_sp, d3h)
size(df)

(15,15,15,4)

In [18]:
# TEST the ForwardDiff for NRL-TB hamiltonian derivatives
# have not  take BOHR into consideration!!

import NRLTB
reload("NRLTB")
BOHR = 0.52917721092 

f = zeros(4)
dh = zeros(15,4)
f = diag( NRLTB.get_os(Float64[ norm(r[:,k]/BOHR) for k = 1:size(r)[2]], NRLTB.C_sp) )
df = NRLTB.get_dos_fd!(r/BOHR, NRLTB.C_sp, dh)/BOHR

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:15
        r[n] += δ
        fh = diag( NRLTB.get_os(Float64[ norm(r[:,k]/BOHR) for k = 1:size(r)[2]], NRLTB.C_sp) )
        dfh[n,:] = ( (fh - f) / δ ) 
        r[n] -= δ
    end
    err = reshape( dfh - df , 15*4)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 4.0105227e-07 
  3 | 4.0680395e-08 
  4 | 4.0729005e-09 
  5 | 4.0472362e-10 
  6 | 1.0239804e-10 
  7 | 1.3664400e-09 
  8 | 9.1380011e-09 
  9 | 1.0525874e-07 
 10 | 1.0801410e-06 
 11 | 9.2483208e-06 
 12 | 1.0814030e-04 
-----------------------------


In [32]:
# test hessian
import NRLTB
reload("NRLTB")

dh = zeros(15,4)
fh = zeros(15,4)
d2h = zeros(15,15,4)
f = NRLTB.get_dos_fd!(r/BOHR, NRLTB.C_sp, dh)
f[:]/=BOHR
df = NRLTB.get_d2os_fd!(r/BOHR, NRLTB.C_sp, d2h)/ (BOHR^2)

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:15
        r[n] += δ
        NRLTB.get_dos_fd!(r/BOHR, NRLTB.C_sp, fh)
        fh[:]/=BOHR
        # NOTE:
        # fh = NRLTB.get_dos_fd(r, NRLTB.C_sp, dh) does not change the value of fh!!
        # print(norm((f-fh)[:],Inf))
        dfh[n,:] = ( ( fh - f) / δ ) 
        r[n] -= δ
    end
    err = reshape( dfh - df , 15*15*4)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")


-----------------------------
  p | error 
----|------------------------
  2 | 1.9129319e-06 
  3 | 1.9358648e-07 
  4 | 1.9381816e-08 
  5 | 1.9384104e-09 
  6 | 1.9381540e-10 
  7 | 1.9296119e-11 
  8 | 5.5911262e-12 
  9 | 3.5547672e-11 
 10 | 5.2343865e-10 
 11 | 4.4868694e-09 
 12 | 3.8876407e-08 
-----------------------------


In [38]:
# test 3rd-order derivative
import NRLTB
reload("NRLTB")

d2h = zeros(15,15,4)
fh = zeros(15,15,4)
d3h = zeros(15,15,15,4)
f = NRLTB.get_d2os_fd!(r/BOHR, NRLTB.C_sp, d2h)
f[:]/=BOHR^2
df = NRLTB.get_d3os_fd!(r/BOHR, NRLTB.C_sp, d3h)/BOHR^3

println("-----------------------------")
println("  p | error ")
println("----|------------------------")
for p = 2:12
    δ = 0.1^p
    dfh = zeros(size(df))
    for n = 1:15
        r[n] += δ
        NRLTB.get_d2os_fd!(r/BOHR, NRLTB.C_sp, fh)
        fh[:]/=BOHR^2
        # NOTE:
        # fh = NRLTB.get_dos_fd(r, NRLTB.C_sp, dh) does not change the value of fh!!
        # print(norm((f-fh)[:],Inf))
        #
        dfh[n,:] = ( ( fh - f) / δ ) 
        r[n] -= δ
    end
    err = reshape( dfh - df , 15*15*15*4)
    @printf(" %2d | %1.7e \n", p, norm(err, Inf))
end
println("-----------------------------")

-----------------------------
  p | error 
----|------------------------
  2 | 7.6259991e-06 
  3 | 7.7210531e-07 
  4 | 7.7306675e-08 
  5 | 7.7316136e-09 
  6 | 7.7303328e-10 
  7 | 7.6623115e-11 
  8 | 2.4445886e-11 
  9 | 1.6705359e-10 
 10 | 2.8504540e-09 
 11 | 2.0812584e-08 
 12 | 2.2410049e-07 
-----------------------------
